In [1]:
import copy
import random

import numpy as np
from PIL import Image
from torchvision import datasets, transforms
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from scipy.spatial.distance import cdist

import inclearn

np.seterr(divide='raise', invalid='raise')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [2]:
EPSILON = 1e-8

In [3]:
seed = 1
print("Set seed", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True  # This will slow down training.

Set seed 1


In [4]:
device = torch.device("cuda:0")

In [5]:
train_dataset = datasets.cifar.CIFAR100("data/", train=True, download=True)
test_dataset = datasets.cifar.CIFAR100("data/", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
x_train, y_train = train_dataset.data, np.array(train_dataset.targets)
x_test, y_test = test_dataset.data, np.array(test_dataset.targets)

In [7]:
class_order = [87,  0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61,
                        14,  8, 86, 84, 96, 18, 24, 32, 45, 88, 11,  4, 67, 69, 66, 77, 47,
                        79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73,  1,
                        28,  6, 46, 62, 82, 53,  9, 31, 75, 38, 63, 33, 74, 27, 22, 36,  3,
                        16, 21, 60, 19, 70, 90, 89, 43,  5, 42, 65, 76, 40, 30, 23, 85,  2,
                        95, 56, 48, 71, 64, 98, 13, 99,  7, 34, 55, 54, 26, 35, 39]

In [8]:
y_train = np.array(list(map(lambda x: class_order.index(x), y_train)))
y_test = np.array(list(map(lambda x: class_order.index(x), y_test)))

In [9]:
class IncDataset(torch.utils.data.Dataset):
    def __init__(self, x, y, train=True):
        self.data_x = x
        self.data_y = y
        
        if train == True:
            self.transforms =  transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5071, 0.4867, 0.4408),
                                     (0.2675, 0.2565, 0.2761)),
            ])
        elif train == False or train is None:
            self.transforms =  transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5071, 0.4867, 0.4408),
                                     (0.2675, 0.2565, 0.2761)),
            ])
        elif train == "flip":
            self.transforms =  transforms.Compose([
                transforms.RandomHorizontalFlip(p=1.0),
                transforms.ToTensor(),
                transforms.Normalize((0.5071, 0.4867, 0.4408),
                                     (0.2675, 0.2565, 0.2761)),
            ])
        else:
            raise Exception
            
    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        x = self.data_x[idx]
        y = self.data_y[idx]
        x = Image.fromarray(x)
        x = self.transforms(x)
        
        return x, y

In [10]:
def select(low_range=0, high_range=0, train=True):
    if train:
        x, y = x_train.copy(), y_train.copy()
    else:
        x, y = x_test.copy(), y_test.copy()
        
    idxes = np.where(np.logical_and(y>=low_range, y<high_range))[0]
    return x[idxes], y[idxes]


def create_dataset(low_range=0, high_range=100, additionnal_x=None, additionnal_y=None):
    test_dataset = IncDataset(*select(low_range=0, high_range=high_range, train=False), train=False)
    
    x, y = select(low_range=low_range, high_range=high_range, train=True)
    if len(additionnal_x) > 0:
        x = np.concatenate((x, np.concatenate(additionnal_x)))
        y = np.concatenate((y, np.concatenate(additionnal_y)))
      
    train_dataset = IncDataset(x, y, train=True)
    
    print("Test: ", len(test_dataset), "Train: ", len(train_dataset))
    return train_dataset, test_dataset

In [11]:
def one_hot(y, nb_classes):
    onehot = torch.zeros(y.shape[0], nb_classes).to(y.device)
    onehot.scatter_(dim=1, index=y.view(-1, 1), value=1.)
    return onehot


def compute_loss(logits, targets, old_targets=None, nb_classes=-1, increment=None):
    targets = one_hot(targets, nb_classes)
    
    if old_targets is None:
        loss =  F.binary_cross_entropy_with_logits(logits, targets)
    else:                                                                                
        new_targets = targets.clone()
        new_targets[..., :-increment] = old_targets
        
        loss = F.binary_cross_entropy_with_logits(logits, new_targets)
        
    if loss < 0 or torch.isnan(loss):
        import pdb; pdb.set_trace()
    
    return loss

In [15]:
def extract_features(model, dataset):
    gen = DataLoader(dataset, shuffle=False, batch_size=256)
    features = []
    targets_all = []
    for inputs, targets in gen:
        features.append(model.extract(inputs.to(model.device)).detach())
        targets_all.append(targets.numpy())
        
    return torch.cat(features), np.concatenate(targets_all)


def extract(model, x, y):
    feat_normal, _ = extract_features(model, IncDataset(x, y, train=None))
    feat_flip, _ = extract_features(model, IncDataset(x, y, train="flip"))
    
    return feat_normal, feat_flip


def select_examplars(features, nb_max):
    D = features.cpu().numpy().T
    D = D / (np.linalg.norm(D, axis=0) + EPSILON)
    mu = np.mean(D, axis=1)
    herding_mat = np.zeros((features.shape[0]))

    w_t = mu
    iter_herding, iter_herding_eff = 0, 0
    
    while not(np.sum(herding_mat!=0)==min(nb_max,features.shape[0])) and iter_herding_eff<1000:
        tmp_t   = np.dot(w_t, D)
        ind_max = np.argmax(tmp_t)
        iter_herding_eff += 1
        if herding_mat[ind_max] == 0:
            herding_mat[ind_max] = 1 + iter_herding
            iter_herding += 1
            
        w_t = w_t + mu - D[:, ind_max]

    return herding_mat


def compute_examplar_mean(feat_norm, feat_flip, herding_mat, nb_max):
    D = feat_norm.cpu().numpy().T
    D = D / (np.linalg.norm(D, axis=0) + EPSILON)
    
    D2 = feat_flip.cpu().numpy().T
    D2 = D2 / (np.linalg.norm(D2, axis=0) + EPSILON)
    
    alph = herding_mat
    alph = (alph > 0) * (alph < nb_max + 1) * 1.
    
    alph_mean = alph / np.sum(alph)
    
    mean = (np.dot(D, alph_mean) + np.dot(D2, alph_mean)) / 2
    mean /= np.linalg.norm(mean)

    return mean, alph


def compute_accuracy(model, test_dataset, class_means):
    features, targets_ = extract_features(model, test_dataset)
    features = features.cpu().numpy()
    
    targets = np.zeros((targets_.shape[0], 100),np.float32)
    targets[range(len(targets_)),targets_.astype('int32')] = 1.
    features  = (features.T / (np.linalg.norm(features.T,axis=0) + EPSILON)).T

    # Compute score for iCaRL
    sqd         = cdist(class_means, features, 'sqeuclidean')
    score_icarl = (-sqd).T

    # Compute the accuracy over the batch
    stat_icarl = [ll in best for ll, best in zip(targets_.astype('int32'), np.argsort(score_icarl, axis=1)[:, -1:])]

    return np.average(stat_icarl)

In [12]:
total = 100

lr = 2.0
scheduling = [49, 63]
gamma = 0.2
nb_epochs = 70
weight_decay = 0.00005
memory = 2000

In [13]:
def fun(increment=10, verbose=False, convnet="resnet34", bias=False):
    print(scheduling, lr)
    model = inclearn.lib.network.BasicNet(convnet, device=device, use_bias=bias)
    old_model = None
    
    memory_x, memory_y = [], []
    herding_mat_all = np.zeros((100, 500))
    class_means = np.zeros((100, model.features_dim))
    all_accs = []

    for task in range(total // increment):
        memory_per_class = memory // ((task + 1) * increment)

        low_range = task * increment
        high_range = (task + 1) * increment

        print("Range: ", low_range, high_range)

        train_dataset, test_dataset = create_dataset(low_range=low_range, high_range=high_range,
                                                     additionnal_x=memory_x, additionnal_y=memory_y)
        train_gen = DataLoader(dataset=train_dataset, shuffle=True, batch_size=128)

        model.add_classes(increment)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, scheduling, gamma=gamma)

        loss_cumul = 0.
        counter = 0
        model.train()
        for epoch in range(nb_epochs):
            scheduler.step()

            for inputs, targets in train_gen:
                counter += 1
                inputs, targets = inputs.to(device), targets.to(device)

                old_targets = None
                if old_model is not None:
                    old_targets = torch.sigmoid(old_model(inputs).detach())

                optimizer.zero_grad()
                logits = model(inputs)
                loss = compute_loss(logits, targets, old_targets, high_range, increment)
                loss.backward()
                optimizer.step()

                loss_cumul += loss.item()

            print("Task {}/{}, Epoch {}/{}, loss: {}.".format(task + 1, total // increment, epoch + 1, nb_epochs, round(loss_cumul / counter, 4)))
        model.eval()
        # Selects memory
        old_model = model.copy().freeze()

        memory_x, memory_y = [], []
        print("Memory stuff up to", high_range)
        for class_idx in range(high_range):
            class_x, class_y = select(low_range=class_idx, high_range=class_idx + 1)
            feat_norm, feat_flip = extract(model, class_x, class_y)

            if class_idx >= low_range:
                print("Finding examplars for", class_idx)
                herding_mat_all[class_idx, :] = select_examplars(feat_norm, memory_per_class)

            examplar_mean, alph = compute_examplar_mean(feat_norm, feat_flip, herding_mat_all[class_idx], memory_per_class)
            memory_x.append(class_x[np.where(alph == 1)[0]])
            memory_y.append(class_y[np.where(alph == 1)[0]])

            class_means[class_idx, :] = examplar_mean

        acc = compute_accuracy(model, test_dataset, class_means)
        all_accs.append(acc)

        print("Task {}/{}, range: 0->{}, acc: {}".format(task + 1, total // increment, high_range, round(acc, 3)))
    
    print(sum(all_accs) / len(all_accs))
    return all_accs

In [14]:
fun(increment=10)

Range:  0 10
Test:  1000 Train:  5000
Epoch 0/70, loss: 0.3906.
Epoch 1/70, loss: 0.3216.
Epoch 2/70, loss: 0.2879.
Epoch 3/70, loss: 0.2682.
Epoch 4/70, loss: 0.2531.
Epoch 5/70, loss: 0.2406.
Epoch 6/70, loss: 0.23.
Epoch 7/70, loss: 0.2223.
Epoch 8/70, loss: 0.2149.
Epoch 9/70, loss: 0.208.
Epoch 10/70, loss: 0.204.
Epoch 11/70, loss: 0.2001.
Epoch 12/70, loss: 0.195.
Epoch 13/70, loss: 0.1901.
Epoch 14/70, loss: 0.1862.
Epoch 15/70, loss: 0.182.
Epoch 16/70, loss: 0.1786.
Epoch 17/70, loss: 0.1744.
Epoch 18/70, loss: 0.1713.
Epoch 19/70, loss: 0.1691.
Epoch 20/70, loss: 0.166.
Epoch 21/70, loss: 0.1629.
Epoch 22/70, loss: 0.1596.
Epoch 23/70, loss: 0.1568.
Epoch 24/70, loss: 0.1539.
Epoch 25/70, loss: 0.1518.
Epoch 26/70, loss: 0.1493.
Epoch 27/70, loss: 0.1471.
Epoch 28/70, loss: 0.1445.
Epoch 29/70, loss: 0.1422.
Epoch 30/70, loss: 0.1399.
Epoch 52/70, loss: 0.1055.
Epoch 53/70, loss: 0.1039.
Epoch 54/70, loss: 0.1023.
Epoch 55/70, loss: 0.1008.
Epoch 56/70, loss: 0.0994.
Epoch 5

Finding examplars for 38
Finding examplars for 39
Task: 3, range: 0->40, acc: 0.63
Range:  40 50
Test:  5000 Train:  7000
Epoch 0/70, loss: 0.0977.
Epoch 1/70, loss: 0.0922.
Epoch 2/70, loss: 0.089.
Epoch 3/70, loss: 0.0867.
Epoch 4/70, loss: 0.0851.
Epoch 5/70, loss: 0.0837.
Epoch 6/70, loss: 0.0825.
Epoch 7/70, loss: 0.0815.
Epoch 8/70, loss: 0.0807.
Epoch 9/70, loss: 0.08.
Epoch 10/70, loss: 0.0794.
Epoch 11/70, loss: 0.0788.
Epoch 12/70, loss: 0.0782.
Epoch 13/70, loss: 0.0778.
Epoch 14/70, loss: 0.0773.
Epoch 15/70, loss: 0.0769.
Epoch 16/70, loss: 0.0765.
Epoch 17/70, loss: 0.0762.
Epoch 18/70, loss: 0.0758.
Epoch 19/70, loss: 0.0755.
Epoch 20/70, loss: 0.0752.
Epoch 21/70, loss: 0.0749.
Epoch 22/70, loss: 0.0747.
Epoch 23/70, loss: 0.0745.
Epoch 24/70, loss: 0.0743.
Epoch 25/70, loss: 0.0741.
Epoch 26/70, loss: 0.0739.
Epoch 27/70, loss: 0.0737.
Epoch 28/70, loss: 0.0735.
Epoch 29/70, loss: 0.0733.
Epoch 30/70, loss: 0.0731.
Epoch 31/70, loss: 0.073.
Epoch 32/70, loss: 0.0728.
E

Epoch 54/70, loss: 0.0717.
Epoch 55/70, loss: 0.0716.
Epoch 56/70, loss: 0.0715.
Epoch 57/70, loss: 0.0714.
Epoch 58/70, loss: 0.0713.
Epoch 59/70, loss: 0.0712.
Epoch 60/70, loss: 0.0711.
Epoch 61/70, loss: 0.071.
Epoch 62/70, loss: 0.0709.
Epoch 63/70, loss: 0.0708.
Epoch 64/70, loss: 0.0707.
Epoch 65/70, loss: 0.0706.
Epoch 66/70, loss: 0.0706.
Epoch 67/70, loss: 0.0705.
Epoch 68/70, loss: 0.0704.
Epoch 69/70, loss: 0.0703.
Memory stuff up to 80
Finding examplars for 70
Finding examplars for 71
Finding examplars for 72
Finding examplars for 73
Finding examplars for 74
Finding examplars for 75
Finding examplars for 76
Finding examplars for 77
Finding examplars for 78
Finding examplars for 79
Task: 7, range: 0->80, acc: 0.482
Range:  80 90
Test:  9000 Train:  7000
Epoch 0/70, loss: 0.0815.
Epoch 1/70, loss: 0.0789.
Epoch 2/70, loss: 0.0773.
Epoch 3/70, loss: 0.0762.
Epoch 4/70, loss: 0.0754.
Epoch 5/70, loss: 0.0746.
Epoch 6/70, loss: 0.074.
Epoch 7/70, loss: 0.0735.
Epoch 8/70, loss:

[0.857,
 0.736,
 0.6866666666666666,
 0.62975,
 0.582,
 0.5448333333333333,
 0.5141428571428571,
 0.481625,
 0.4593333333333333,
 0.4155]

In [15]:
fun(increment=50)

Range:  0 50
Test:  5000 Train:  25000
Epoch 0/70, loss: 0.0981.
Epoch 1/70, loss: 0.0889.
Epoch 2/70, loss: 0.083.
Epoch 3/70, loss: 0.0783.
Epoch 4/70, loss: 0.0745.
Epoch 5/70, loss: 0.0712.
Epoch 6/70, loss: 0.0683.
Epoch 7/70, loss: 0.0658.
Epoch 8/70, loss: 0.0636.
Epoch 9/70, loss: 0.0617.
Epoch 10/70, loss: 0.0599.
Epoch 11/70, loss: 0.0582.
Epoch 12/70, loss: 0.0567.
Epoch 13/70, loss: 0.0553.
Epoch 14/70, loss: 0.054.
Epoch 15/70, loss: 0.0528.
Epoch 16/70, loss: 0.0516.
Epoch 17/70, loss: 0.0505.
Epoch 18/70, loss: 0.0495.
Epoch 19/70, loss: 0.0485.
Epoch 20/70, loss: 0.0476.
Epoch 21/70, loss: 0.0467.
Epoch 22/70, loss: 0.0458.
Epoch 23/70, loss: 0.045.
Epoch 24/70, loss: 0.0442.
Epoch 25/70, loss: 0.0435.
Epoch 26/70, loss: 0.0428.
Epoch 27/70, loss: 0.0421.
Epoch 28/70, loss: 0.0414.
Epoch 29/70, loss: 0.0408.
Epoch 30/70, loss: 0.0402.
Epoch 31/70, loss: 0.0396.
Epoch 32/70, loss: 0.039.
Epoch 33/70, loss: 0.0385.
Epoch 34/70, loss: 0.0379.
Epoch 35/70, loss: 0.0374.
Epo

[0.7376, 0.6277]

In [20]:
icarl_resnet34_2 = _

In [22]:
icarl_rebuffi_10 = fun(increment=10, convnet="rebuffi")
icarl_rebuffi_10

Range:  0 10
Test:  1000 Train:  5000
Epoch 0/70, loss: 0.3996.
Epoch 1/70, loss: 0.3595.
Epoch 2/70, loss: 0.3427.
Epoch 3/70, loss: 0.3295.
Epoch 4/70, loss: 0.3185.
Epoch 5/70, loss: 0.309.
Epoch 6/70, loss: 0.3007.
Epoch 7/70, loss: 0.2936.
Epoch 8/70, loss: 0.2873.
Epoch 9/70, loss: 0.2812.
Epoch 10/70, loss: 0.275.
Epoch 11/70, loss: 0.2698.
Epoch 12/70, loss: 0.2648.
Epoch 13/70, loss: 0.2597.
Epoch 14/70, loss: 0.255.
Epoch 43/70, loss: 0.1655.
Epoch 44/70, loss: 0.1635.
Epoch 45/70, loss: 0.1617.
Epoch 46/70, loss: 0.1598.
Epoch 47/70, loss: 0.1581.
Epoch 48/70, loss: 0.1564.
Epoch 49/70, loss: 0.1546.
Epoch 50/70, loss: 0.1524.
Epoch 51/70, loss: 0.1503.
Epoch 52/70, loss: 0.1482.
Epoch 53/70, loss: 0.1461.
Epoch 54/70, loss: 0.1441.
Epoch 55/70, loss: 0.1421.
Epoch 56/70, loss: 0.1401.
Epoch 57/70, loss: 0.1382.
Epoch 58/70, loss: 0.1364.
Epoch 59/70, loss: 0.1346.
Epoch 60/70, loss: 0.1328.
Epoch 61/70, loss: 0.1311.
Epoch 62/70, loss: 0.1294.
Epoch 63/70, loss: 0.1278.
Epo

Epoch 2/70, loss: 0.0934.
Epoch 3/70, loss: 0.0915.
Epoch 4/70, loss: 0.09.
Epoch 5/70, loss: 0.0888.
Epoch 6/70, loss: 0.088.
Epoch 7/70, loss: 0.0873.
Epoch 8/70, loss: 0.0866.
Epoch 9/70, loss: 0.086.
Epoch 10/70, loss: 0.0855.
Epoch 11/70, loss: 0.085.
Epoch 12/70, loss: 0.0846.
Epoch 13/70, loss: 0.0842.
Epoch 14/70, loss: 0.0839.
Epoch 15/70, loss: 0.0836.
Epoch 16/70, loss: 0.0833.
Epoch 17/70, loss: 0.083.
Epoch 18/70, loss: 0.0827.
Epoch 19/70, loss: 0.0825.
Epoch 20/70, loss: 0.0822.
Epoch 21/70, loss: 0.082.
Epoch 22/70, loss: 0.0818.
Epoch 23/70, loss: 0.0815.
Epoch 24/70, loss: 0.0813.
Epoch 25/70, loss: 0.0811.
Epoch 26/70, loss: 0.081.
Epoch 27/70, loss: 0.0808.
Epoch 28/70, loss: 0.0806.
Epoch 29/70, loss: 0.0805.
Epoch 30/70, loss: 0.0803.
Epoch 31/70, loss: 0.0802.
Epoch 32/70, loss: 0.0801.
Epoch 33/70, loss: 0.0799.
Epoch 34/70, loss: 0.0798.
Epoch 35/70, loss: 0.0797.
Epoch 36/70, loss: 0.0795.
Epoch 37/70, loss: 0.0794.
Epoch 38/70, loss: 0.0793.
Epoch 39/70, loss

Epoch 60/70, loss: 0.0759.
Epoch 61/70, loss: 0.0758.
Epoch 62/70, loss: 0.0757.
Epoch 63/70, loss: 0.0756.
Epoch 64/70, loss: 0.0755.
Epoch 65/70, loss: 0.0754.
Epoch 66/70, loss: 0.0753.
Epoch 67/70, loss: 0.0752.
Epoch 68/70, loss: 0.0751.
Epoch 69/70, loss: 0.075.
Memory stuff up to 80
Finding examplars for 70
Finding examplars for 71
Finding examplars for 72
Finding examplars for 73
Finding examplars for 74
Finding examplars for 75
Finding examplars for 76
Finding examplars for 77
Finding examplars for 78
Finding examplars for 79
Task: 7, range: 0->80, acc: 0.512
Range:  80 90
Test:  9000 Train:  7000
Epoch 0/70, loss: 0.0882.
Epoch 1/70, loss: 0.0837.
Epoch 2/70, loss: 0.0816.
Epoch 3/70, loss: 0.0803.
Epoch 4/70, loss: 0.0795.
Epoch 5/70, loss: 0.0788.
Epoch 6/70, loss: 0.0782.
Epoch 7/70, loss: 0.0778.
Epoch 8/70, loss: 0.0773.
Epoch 9/70, loss: 0.077.
Epoch 10/70, loss: 0.0767.
Epoch 11/70, loss: 0.0764.
Epoch 12/70, loss: 0.0762.
Epoch 13/70, loss: 0.0759.
Epoch 14/70, loss: 

[0.86,
 0.7485,
 0.6846666666666666,
 0.6375,
 0.5932,
 0.5748333333333333,
 0.5444285714285715,
 0.51175,
 0.4958888888888889,
 0.4701]

In [59]:
icarl_temp2betterschedule_10 = fun(increment=10, convnet="rebuffi", bias=True)
icarl_temp2betterschedule_10

[49, 63] 2.0
Range:  0 10
Test:  1000 Train:  5000
Epoch 0/70, loss: 0.3739.
Epoch 1/70, loss: 0.3381.
Epoch 2/70, loss: 0.3233.
Epoch 3/70, loss: 0.3147.
Epoch 4/70, loss: 0.3074.
Epoch 5/70, loss: 0.3013.
Epoch 6/70, loss: 0.2956.
Epoch 7/70, loss: 0.2901.
Epoch 8/70, loss: 0.2848.
Epoch 9/70, loss: 0.28.
Epoch 10/70, loss: 0.2748.
Epoch 11/70, loss: 0.2703.
Epoch 12/70, loss: 0.2656.
Epoch 13/70, loss: 0.262.
Epoch 14/70, loss: 0.2574.
Epoch 15/70, loss: 0.2537.
Epoch 16/70, loss: 0.2494.
Epoch 17/70, loss: 0.2462.
Epoch 18/70, loss: 0.2426.
Epoch 19/70, loss: 0.2388.
Epoch 20/70, loss: 0.2349.
Epoch 21/70, loss: 0.2312.
Epoch 22/70, loss: 0.228.
Epoch 23/70, loss: 0.2248.
Epoch 24/70, loss: 0.222.
Epoch 25/70, loss: 0.2192.
Epoch 26/70, loss: 0.2161.
Epoch 27/70, loss: 0.2134.
Epoch 28/70, loss: 0.2109.
Epoch 29/70, loss: 0.2085.
Epoch 30/70, loss: 0.206.
Epoch 31/70, loss: 0.2036.
Epoch 32/70, loss: 0.2014.
Epoch 33/70, loss: 0.1989.
Epoch 34/70, loss: 0.1967.
Epoch 35/70, loss: 0

Epoch 57/70, loss: 0.1075.
Epoch 58/70, loss: 0.1072.
Epoch 59/70, loss: 0.1068.
Epoch 60/70, loss: 0.1065.
Epoch 61/70, loss: 0.1062.
Epoch 62/70, loss: 0.1058.
Epoch 63/70, loss: 0.1055.
Epoch 64/70, loss: 0.1052.
Epoch 65/70, loss: 0.1048.
Epoch 66/70, loss: 0.1045.
Epoch 67/70, loss: 0.1042.
Epoch 68/70, loss: 0.1039.
Epoch 69/70, loss: 0.1036.
Memory stuff up to 40
Finding examplars for 30
Finding examplars for 31
Finding examplars for 32
Finding examplars for 33
Finding examplars for 34
Finding examplars for 35
Finding examplars for 36
Finding examplars for 37
Finding examplars for 38
Finding examplars for 39
Task: 3, range: 0->40, acc: 0.671
Range:  40 50
Test:  5000 Train:  7000
Epoch 0/70, loss: 0.1332.
Epoch 1/70, loss: 0.1245.
Epoch 2/70, loss: 0.121.
Epoch 3/70, loss: 0.1189.
Epoch 4/70, loss: 0.1177.
Epoch 5/70, loss: 0.1166.
Epoch 6/70, loss: 0.1157.
Epoch 7/70, loss: 0.1151.
Epoch 8/70, loss: 0.1146.
Epoch 9/70, loss: 0.1141.
Epoch 10/70, loss: 0.1137.
Epoch 11/70, loss:

Epoch 32/70, loss: 0.1075.
Epoch 33/70, loss: 0.1074.
Epoch 34/70, loss: 0.1074.
Epoch 35/70, loss: 0.1073.
Epoch 36/70, loss: 0.1073.
Epoch 37/70, loss: 0.1072.
Epoch 38/70, loss: 0.1072.
Epoch 39/70, loss: 0.1071.
Epoch 40/70, loss: 0.1071.
Epoch 41/70, loss: 0.107.
Epoch 42/70, loss: 0.107.
Epoch 43/70, loss: 0.1069.
Epoch 44/70, loss: 0.1069.
Epoch 45/70, loss: 0.1069.
Epoch 46/70, loss: 0.1068.
Epoch 47/70, loss: 0.1068.
Epoch 48/70, loss: 0.1068.
Epoch 49/70, loss: 0.1066.
Epoch 50/70, loss: 0.1065.
Epoch 51/70, loss: 0.1063.
Epoch 52/70, loss: 0.1061.
Epoch 53/70, loss: 0.1059.
Epoch 54/70, loss: 0.1058.
Epoch 55/70, loss: 0.1056.
Epoch 56/70, loss: 0.1054.
Epoch 57/70, loss: 0.1053.
Epoch 58/70, loss: 0.1051.
Epoch 59/70, loss: 0.105.
Epoch 60/70, loss: 0.1048.
Epoch 61/70, loss: 0.1047.
Epoch 62/70, loss: 0.1045.
Epoch 63/70, loss: 0.1044.
Epoch 64/70, loss: 0.1042.
Epoch 65/70, loss: 0.1041.
Epoch 66/70, loss: 0.1039.
Epoch 67/70, loss: 0.1038.
Epoch 68/70, loss: 0.1036.
Epoc

[0.856,
 0.7635,
 0.717,
 0.67125,
 0.6188,
 0.581,
 0.5534285714285714,
 0.521875,
 0.4891111111111111,
 0.4573]

In [79]:
icarl_specialseed_10 = fun(increment=10, convnet="rebuffi", bias=True)
icarl_specialseed_10

[49, 63] 2.0
Range:  0 10
Test:  1000 Train:  5000
Task 1/10, Epoch 1/70, loss: 0.3802.
Task 1/10, Epoch 2/70, loss: 0.3389.
Task 1/10, Epoch 3/70, loss: 0.3233.
Task 1/10, Epoch 4/70, loss: 0.3132.
Task 1/10, Epoch 5/70, loss: 0.3046.
Task 1/10, Epoch 6/70, loss: 0.2973.
Task 1/10, Epoch 7/70, loss: 0.2895.
Task 1/10, Epoch 8/70, loss: 0.282.
Task 1/10, Epoch 9/70, loss: 0.2752.
Task 1/10, Epoch 10/70, loss: 0.2686.
Task 1/10, Epoch 11/70, loss: 0.2629.
Task 1/10, Epoch 12/70, loss: 0.2579.
Task 1/10, Epoch 13/70, loss: 0.2529.
Task 1/10, Epoch 14/70, loss: 0.2475.
Task 1/10, Epoch 15/70, loss: 0.2433.
Task 1/10, Epoch 16/70, loss: 0.2388.
Task 1/10, Epoch 17/70, loss: 0.2342.
Task 1/10, Epoch 18/70, loss: 0.2301.
Task 1/10, Epoch 19/70, loss: 0.2256.
Task 1/10, Epoch 20/70, loss: 0.2214.
Task 1/10, Epoch 21/70, loss: 0.2176.
Task 1/10, Epoch 22/70, loss: 0.2137.
Task 1/10, Epoch 23/70, loss: 0.21.
Task 1/10, Epoch 24/70, loss: 0.2072.
Task 1/10, Epoch 25/70, loss: 0.2038.
Task 1/10, 

Task 3/10, Epoch 59/70, loss: 0.1058.
Task 3/10, Epoch 60/70, loss: 0.1054.
Task 3/10, Epoch 61/70, loss: 0.1049.
Task 3/10, Epoch 62/70, loss: 0.1045.
Task 3/10, Epoch 63/70, loss: 0.1041.
Task 3/10, Epoch 64/70, loss: 0.1037.
Task 3/10, Epoch 65/70, loss: 0.1032.
Task 3/10, Epoch 66/70, loss: 0.1028.
Task 3/10, Epoch 67/70, loss: 0.1023.
Task 3/10, Epoch 68/70, loss: 0.1019.
Task 3/10, Epoch 69/70, loss: 0.1015.
Task 3/10, Epoch 70/70, loss: 0.1011.
Memory stuff up to 30
Finding examplars for 20
Finding examplars for 21
Finding examplars for 22
Finding examplars for 23
Finding examplars for 24
Finding examplars for 25
Finding examplars for 26
Finding examplars for 27
Finding examplars for 28
Finding examplars for 29
Task: 2, range: 0->30, acc: 0.744
Range:  30 40
Test:  4000 Train:  6980
Task 4/10, Epoch 1/70, loss: 0.1354.
Task 4/10, Epoch 2/70, loss: 0.1253.
Task 4/10, Epoch 3/70, loss: 0.121.
Task 4/10, Epoch 4/70, loss: 0.1184.
Task 4/10, Epoch 5/70, loss: 0.1165.
Task 4/10, Epoc

Task 6/10, Epoch 39/70, loss: 0.1001.
Task 6/10, Epoch 40/70, loss: 0.1001.
Task 6/10, Epoch 41/70, loss: 0.1.
Task 6/10, Epoch 42/70, loss: 0.0999.
Task 6/10, Epoch 43/70, loss: 0.0999.
Task 6/10, Epoch 44/70, loss: 0.0998.
Task 6/10, Epoch 45/70, loss: 0.0998.
Task 6/10, Epoch 46/70, loss: 0.0997.
Task 6/10, Epoch 47/70, loss: 0.0997.
Task 6/10, Epoch 48/70, loss: 0.0997.
Task 6/10, Epoch 49/70, loss: 0.0996.
Task 6/10, Epoch 50/70, loss: 0.0995.
Task 6/10, Epoch 51/70, loss: 0.0993.
Task 6/10, Epoch 52/70, loss: 0.0991.
Task 6/10, Epoch 53/70, loss: 0.0989.
Task 6/10, Epoch 54/70, loss: 0.0987.
Task 6/10, Epoch 55/70, loss: 0.0985.
Task 6/10, Epoch 56/70, loss: 0.0983.
Task 6/10, Epoch 57/70, loss: 0.0981.
Task 6/10, Epoch 58/70, loss: 0.0979.
Task 6/10, Epoch 59/70, loss: 0.0977.
Task 6/10, Epoch 60/70, loss: 0.0975.
Task 6/10, Epoch 61/70, loss: 0.0973.
Task 6/10, Epoch 62/70, loss: 0.0972.
Task 6/10, Epoch 63/70, loss: 0.097.
Task 6/10, Epoch 64/70, loss: 0.0968.
Task 6/10, Epoch

Task 9/10, Epoch 19/70, loss: 0.105.
Task 9/10, Epoch 20/70, loss: 0.1049.
Task 9/10, Epoch 21/70, loss: 0.1048.
Task 9/10, Epoch 22/70, loss: 0.1048.
Task 9/10, Epoch 23/70, loss: 0.1047.
Task 9/10, Epoch 24/70, loss: 0.1046.
Task 9/10, Epoch 25/70, loss: 0.1045.
Task 9/10, Epoch 26/70, loss: 0.1045.
Task 9/10, Epoch 27/70, loss: 0.1044.
Task 9/10, Epoch 28/70, loss: 0.1044.
Task 9/10, Epoch 29/70, loss: 0.1043.
Task 9/10, Epoch 30/70, loss: 0.1042.
Task 9/10, Epoch 31/70, loss: 0.1042.
Task 9/10, Epoch 32/70, loss: 0.1041.
Task 9/10, Epoch 33/70, loss: 0.1041.
Task 9/10, Epoch 34/70, loss: 0.104.
Task 9/10, Epoch 35/70, loss: 0.104.
Task 9/10, Epoch 36/70, loss: 0.1039.
Task 9/10, Epoch 37/70, loss: 0.1039.
Task 9/10, Epoch 38/70, loss: 0.1039.
Task 9/10, Epoch 39/70, loss: 0.1038.
Task 9/10, Epoch 40/70, loss: 0.1038.
Task 9/10, Epoch 41/70, loss: 0.1038.
Task 9/10, Epoch 42/70, loss: 0.1037.
Task 9/10, Epoch 43/70, loss: 0.1037.
Task 9/10, Epoch 44/70, loss: 0.1037.
Task 9/10, Epoc

[0.887,
 0.806,
 0.7436666666666667,
 0.672,
 0.6228,
 0.6098333333333333,
 0.5674285714285714,
 0.53725,
 0.49844444444444447,
 0.4509]

In [16]:
icarl_specialseed_10 = fun(increment=10, convnet="rebuffi", bias=True)
icarl_specialseed_10

[49, 63] 2.0
Range:  0 10
Test:  1000 Train:  5000
Task 1/10, Epoch 1/70, loss: 0.3887.
Task 1/10, Epoch 2/70, loss: 0.3466.
Task 1/10, Epoch 3/70, loss: 0.3254.
Task 1/10, Epoch 4/70, loss: 0.3128.
Task 1/10, Epoch 5/70, loss: 0.3005.
Task 1/10, Epoch 6/70, loss: 0.2896.
Task 1/10, Epoch 7/70, loss: 0.2796.
Task 1/10, Epoch 8/70, loss: 0.2701.
Task 1/10, Epoch 9/70, loss: 0.2626.
Task 1/10, Epoch 10/70, loss: 0.2549.
Task 1/10, Epoch 11/70, loss: 0.2475.
Task 1/10, Epoch 12/70, loss: 0.2408.
Task 1/10, Epoch 13/70, loss: 0.2349.
Task 1/10, Epoch 14/70, loss: 0.2307.
Task 1/10, Epoch 15/70, loss: 0.2255.
Task 1/10, Epoch 16/70, loss: 0.2201.
Task 1/10, Epoch 17/70, loss: 0.2155.
Task 1/10, Epoch 18/70, loss: 0.2107.
Task 1/10, Epoch 19/70, loss: 0.2069.
Task 1/10, Epoch 20/70, loss: 0.2038.
Task 1/10, Epoch 21/70, loss: 0.1999.
Task 1/10, Epoch 22/70, loss: 0.196.
Task 1/10, Epoch 23/70, loss: 0.193.
Task 1/10, Epoch 24/70, loss: 0.19.
Task 1/10, Epoch 25/70, loss: 0.1868.
Task 1/10, E

Task 3/10, Epoch 59/70, loss: 0.1043.
Task 3/10, Epoch 60/70, loss: 0.1038.
Task 3/10, Epoch 61/70, loss: 0.1034.
Task 3/10, Epoch 62/70, loss: 0.1029.
Task 3/10, Epoch 63/70, loss: 0.1025.
Task 3/10, Epoch 64/70, loss: 0.102.
Task 3/10, Epoch 65/70, loss: 0.1016.
Task 3/10, Epoch 66/70, loss: 0.1011.
Task 3/10, Epoch 67/70, loss: 0.1007.
Task 3/10, Epoch 68/70, loss: 0.1003.
Task 3/10, Epoch 69/70, loss: 0.0998.
Task 3/10, Epoch 70/70, loss: 0.0994.
Memory stuff up to 30
Finding examplars for 20
Finding examplars for 21
Finding examplars for 22
Finding examplars for 23
Finding examplars for 24
Finding examplars for 25
Finding examplars for 26
Finding examplars for 27
Finding examplars for 28
Finding examplars for 29
Task 3/10, range: 0->30, acc: 0.735
Range:  30 40
Test:  4000 Train:  6980
Task 4/10, Epoch 1/70, loss: 0.1338.
Task 4/10, Epoch 2/70, loss: 0.1245.
Task 4/10, Epoch 3/70, loss: 0.1205.
Task 4/10, Epoch 4/70, loss: 0.1178.
Task 4/10, Epoch 5/70, loss: 0.116.
Task 4/10, Epo

Task 6/10, Epoch 39/70, loss: 0.1007.
Task 6/10, Epoch 40/70, loss: 0.1006.
Task 6/10, Epoch 41/70, loss: 0.1005.
Task 6/10, Epoch 42/70, loss: 0.1005.
Task 6/10, Epoch 43/70, loss: 0.1004.
Task 6/10, Epoch 44/70, loss: 0.1004.
Task 6/10, Epoch 45/70, loss: 0.1004.
Task 6/10, Epoch 46/70, loss: 0.1003.
Task 6/10, Epoch 47/70, loss: 0.1003.
Task 6/10, Epoch 48/70, loss: 0.1002.
Task 6/10, Epoch 49/70, loss: 0.1002.
Task 6/10, Epoch 50/70, loss: 0.1.
Task 6/10, Epoch 51/70, loss: 0.0998.
Task 6/10, Epoch 52/70, loss: 0.0996.
Task 6/10, Epoch 53/70, loss: 0.0994.
Task 6/10, Epoch 54/70, loss: 0.0992.
Task 6/10, Epoch 55/70, loss: 0.099.
Task 6/10, Epoch 56/70, loss: 0.0988.
Task 6/10, Epoch 57/70, loss: 0.0986.
Task 6/10, Epoch 58/70, loss: 0.0984.
Task 6/10, Epoch 59/70, loss: 0.0982.
Task 6/10, Epoch 60/70, loss: 0.098.
Task 6/10, Epoch 61/70, loss: 0.0979.
Task 6/10, Epoch 62/70, loss: 0.0977.
Task 6/10, Epoch 63/70, loss: 0.0975.
Task 6/10, Epoch 64/70, loss: 0.0973.
Task 6/10, Epoch 

Task 9/10, Epoch 19/70, loss: 0.1028.
Task 9/10, Epoch 20/70, loss: 0.1027.
Task 9/10, Epoch 21/70, loss: 0.1026.
Task 9/10, Epoch 22/70, loss: 0.1025.
Task 9/10, Epoch 23/70, loss: 0.1025.
Task 9/10, Epoch 24/70, loss: 0.1024.
Task 9/10, Epoch 25/70, loss: 0.1023.
Task 9/10, Epoch 26/70, loss: 0.1023.
Task 9/10, Epoch 27/70, loss: 0.1022.
Task 9/10, Epoch 28/70, loss: 0.1022.
Task 9/10, Epoch 29/70, loss: 0.1021.
Task 9/10, Epoch 30/70, loss: 0.1021.
Task 9/10, Epoch 31/70, loss: 0.102.
Task 9/10, Epoch 32/70, loss: 0.102.
Task 9/10, Epoch 33/70, loss: 0.1019.
Task 9/10, Epoch 34/70, loss: 0.1019.
Task 9/10, Epoch 35/70, loss: 0.1018.
Task 9/10, Epoch 36/70, loss: 0.1018.
Task 9/10, Epoch 37/70, loss: 0.1018.
Task 9/10, Epoch 38/70, loss: 0.1017.
Task 9/10, Epoch 39/70, loss: 0.1017.
Task 9/10, Epoch 40/70, loss: 0.1017.
Task 9/10, Epoch 41/70, loss: 0.1016.
Task 9/10, Epoch 42/70, loss: 0.1016.
Task 9/10, Epoch 43/70, loss: 0.1016.
Task 9/10, Epoch 44/70, loss: 0.1015.
Task 9/10, Epo

[0.891,
 0.8035,
 0.7353333333333333,
 0.66225,
 0.631,
 0.6045,
 0.5537142857142857,
 0.533375,
 0.493,
 0.4523]

In [81]:
icarl_specialseed_2_10 = fun(increment=10, convnet="rebuffi", bias=True)
icarl_specialseed_2_10

[49, 63] 2.0
Range:  0 10
Test:  1000 Train:  5000
Task 1/10, Epoch 1/70, loss: 0.3693.
Task 1/10, Epoch 2/70, loss: 0.3179.
Task 1/10, Epoch 3/70, loss: 0.2944.
Task 1/10, Epoch 4/70, loss: 0.2782.
Task 1/10, Epoch 5/70, loss: 0.2671.
Task 1/10, Epoch 6/70, loss: 0.2564.
Task 1/10, Epoch 7/70, loss: 0.2467.
Task 1/10, Epoch 8/70, loss: 0.2388.
Task 1/10, Epoch 9/70, loss: 0.2322.
Task 1/10, Epoch 10/70, loss: 0.2259.
Task 1/10, Epoch 11/70, loss: 0.2199.
Task 1/10, Epoch 12/70, loss: 0.2148.
Task 1/10, Epoch 13/70, loss: 0.2095.
Task 1/10, Epoch 14/70, loss: 0.2044.
Task 1/10, Epoch 15/70, loss: 0.2001.
Task 1/10, Epoch 16/70, loss: 0.1957.
Task 1/10, Epoch 17/70, loss: 0.191.
Task 1/10, Epoch 18/70, loss: 0.1871.
Task 1/10, Epoch 19/70, loss: 0.1833.
Task 1/10, Epoch 20/70, loss: 0.1795.
Task 1/10, Epoch 21/70, loss: 0.1756.
Task 1/10, Epoch 22/70, loss: 0.1722.
Task 1/10, Epoch 23/70, loss: 0.1692.
Task 1/10, Epoch 24/70, loss: 0.1663.
Task 1/10, Epoch 25/70, loss: 0.1636.
Task 1/10

Task 3/10, Epoch 59/70, loss: 0.0784.
Task 3/10, Epoch 60/70, loss: 0.0781.
Task 3/10, Epoch 61/70, loss: 0.0777.
Task 3/10, Epoch 62/70, loss: 0.0774.
Task 3/10, Epoch 63/70, loss: 0.0771.
Task 3/10, Epoch 64/70, loss: 0.0767.
Task 3/10, Epoch 65/70, loss: 0.0764.
Task 3/10, Epoch 66/70, loss: 0.0761.
Task 3/10, Epoch 67/70, loss: 0.0758.
Task 3/10, Epoch 68/70, loss: 0.0755.
Task 3/10, Epoch 69/70, loss: 0.0752.
Task 3/10, Epoch 70/70, loss: 0.075.
Memory stuff up to 30
Finding examplars for 20
Finding examplars for 21
Finding examplars for 22
Finding examplars for 23
Finding examplars for 24
Finding examplars for 25
Finding examplars for 26
Finding examplars for 27
Finding examplars for 28
Finding examplars for 29
Task: 2, range: 0->30, acc: 0.712
Range:  30 40
Test:  4000 Train:  6980
Task 4/10, Epoch 1/70, loss: 0.1183.
Task 4/10, Epoch 2/70, loss: 0.107.
Task 4/10, Epoch 3/70, loss: 0.1021.
Task 4/10, Epoch 4/70, loss: 0.099.
Task 4/10, Epoch 5/70, loss: 0.0968.
Task 4/10, Epoch 

Task 6/10, Epoch 39/70, loss: 0.0751.
Task 6/10, Epoch 40/70, loss: 0.075.
Task 6/10, Epoch 41/70, loss: 0.0749.
Task 6/10, Epoch 42/70, loss: 0.0749.
Task 6/10, Epoch 43/70, loss: 0.0748.
Task 6/10, Epoch 44/70, loss: 0.0747.
Task 6/10, Epoch 45/70, loss: 0.0746.
Task 6/10, Epoch 46/70, loss: 0.0746.
Task 6/10, Epoch 47/70, loss: 0.0745.
Task 6/10, Epoch 48/70, loss: 0.0744.
Task 6/10, Epoch 49/70, loss: 0.0744.
Task 6/10, Epoch 50/70, loss: 0.0742.
Task 6/10, Epoch 51/70, loss: 0.0741.
Task 6/10, Epoch 52/70, loss: 0.0739.
Task 6/10, Epoch 53/70, loss: 0.0738.
Task 6/10, Epoch 54/70, loss: 0.0737.
Task 6/10, Epoch 55/70, loss: 0.0735.
Task 6/10, Epoch 56/70, loss: 0.0734.
Task 6/10, Epoch 57/70, loss: 0.0733.
Task 6/10, Epoch 58/70, loss: 0.0731.
Task 6/10, Epoch 59/70, loss: 0.073.
Task 6/10, Epoch 60/70, loss: 0.0729.
Task 6/10, Epoch 61/70, loss: 0.0728.
Task 6/10, Epoch 62/70, loss: 0.0726.
Task 6/10, Epoch 63/70, loss: 0.0725.
Task 6/10, Epoch 64/70, loss: 0.0724.
Task 6/10, Epo

Task 9/10, Epoch 19/70, loss: 0.081.
Task 9/10, Epoch 20/70, loss: 0.0808.
Task 9/10, Epoch 21/70, loss: 0.0807.
Task 9/10, Epoch 22/70, loss: 0.0805.
Task 9/10, Epoch 23/70, loss: 0.0805.
Task 9/10, Epoch 24/70, loss: 0.0803.
Task 9/10, Epoch 25/70, loss: 0.0802.
Task 9/10, Epoch 26/70, loss: 0.0801.
Task 9/10, Epoch 27/70, loss: 0.08.
Task 9/10, Epoch 28/70, loss: 0.0799.
Task 9/10, Epoch 29/70, loss: 0.0798.
Task 9/10, Epoch 30/70, loss: 0.0797.
Task 9/10, Epoch 31/70, loss: 0.0797.
Task 9/10, Epoch 32/70, loss: 0.0796.
Task 9/10, Epoch 33/70, loss: 0.0795.
Task 9/10, Epoch 34/70, loss: 0.0794.
Task 9/10, Epoch 35/70, loss: 0.0794.
Task 9/10, Epoch 36/70, loss: 0.0793.
Task 9/10, Epoch 37/70, loss: 0.0792.
Task 9/10, Epoch 38/70, loss: 0.0791.
Task 9/10, Epoch 39/70, loss: 0.0791.
Task 9/10, Epoch 40/70, loss: 0.079.
Task 9/10, Epoch 41/70, loss: 0.079.
Task 9/10, Epoch 42/70, loss: 0.0789.
Task 9/10, Epoch 43/70, loss: 0.0789.
Task 9/10, Epoch 44/70, loss: 0.0788.
Task 9/10, Epoch 

[0.896,
 0.7925,
 0.7116666666666667,
 0.6395,
 0.5968,
 0.5886666666666667,
 0.5444285714285715,
 0.526625,
 0.5106666666666667,
 0.4662]

In [ ]:
icarl_specialseed_2 = fun(increment=2, convnet="rebuffi", bias=True)
icarl_specialseed_2

[49, 63] 2.0
Range:  0 2
Test:  200 Train:  1000
Task 1/50, Epoch 1/70, loss: 4.3141.
Task 1/50, Epoch 2/70, loss: 2.4184.
Task 1/50, Epoch 3/70, loss: 1.721.
Task 1/50, Epoch 4/70, loss: 1.3636.
Task 1/50, Epoch 5/70, loss: 1.1464.
Task 1/50, Epoch 6/70, loss: 0.9974.
Task 1/50, Epoch 7/70, loss: 0.8892.
Task 1/50, Epoch 8/70, loss: 0.8031.
Task 1/50, Epoch 9/70, loss: 0.7391.
Task 1/50, Epoch 10/70, loss: 0.6854.
Task 1/50, Epoch 11/70, loss: 0.6478.
Task 1/50, Epoch 12/70, loss: 0.6124.
Task 1/50, Epoch 13/70, loss: 0.5794.
Task 1/50, Epoch 14/70, loss: 0.5493.
Task 1/50, Epoch 15/70, loss: 0.5235.
Task 1/50, Epoch 16/70, loss: 0.5046.
Task 1/50, Epoch 17/70, loss: 0.4851.
Task 1/50, Epoch 18/70, loss: 0.4666.
Task 1/50, Epoch 19/70, loss: 0.4499.
Task 1/50, Epoch 20/70, loss: 0.4345.
Task 1/50, Epoch 21/70, loss: 0.4205.
Task 1/50, Epoch 22/70, loss: 0.4086.
Task 1/50, Epoch 23/70, loss: 0.3973.
Task 1/50, Epoch 24/70, loss: 0.3864.
Task 1/50, Epoch 25/70, loss: 0.3764.
Task 1/50, 

Task 3/50, Epoch 70/70, loss: 0.373.
Memory stuff up to 6
Finding examplars for 4
Finding examplars for 5
Task 3/50, range: 0->6, acc: 0.432
Range:  6 8
Test:  800 Train:  1820
Task 4/50, Epoch 1/70, loss: 0.4391.
Task 4/50, Epoch 2/70, loss: 0.4351.
Task 4/50, Epoch 3/70, loss: 0.4341.
Task 4/50, Epoch 4/70, loss: 0.4319.
Task 4/50, Epoch 5/70, loss: 0.4319.
Task 4/50, Epoch 6/70, loss: 0.4313.
Task 4/50, Epoch 7/70, loss: 0.4303.
Task 4/50, Epoch 8/70, loss: 0.4294.
Task 4/50, Epoch 9/70, loss: 0.428.
Task 4/50, Epoch 10/70, loss: 0.4277.
Task 4/50, Epoch 11/70, loss: 0.4271.
Task 4/50, Epoch 12/70, loss: 0.4263.
Task 4/50, Epoch 13/70, loss: 0.4253.
Task 4/50, Epoch 14/70, loss: 0.4245.
Task 4/50, Epoch 15/70, loss: 0.4238.
Task 4/50, Epoch 16/70, loss: 0.4228.
Task 4/50, Epoch 17/70, loss: 0.4223.
Task 4/50, Epoch 18/70, loss: 0.4218.
Task 4/50, Epoch 19/70, loss: 0.4213.
Task 4/50, Epoch 20/70, loss: 0.4208.
Task 4/50, Epoch 21/70, loss: 0.4204.
Task 4/50, Epoch 22/70, loss: 0.419

Task 6/50, Epoch 66/70, loss: 0.3658.
Task 6/50, Epoch 67/70, loss: 0.3655.
Task 6/50, Epoch 68/70, loss: 0.3653.
Task 6/50, Epoch 69/70, loss: 0.3651.
Task 6/50, Epoch 70/70, loss: 0.3649.
Memory stuff up to 12
Finding examplars for 10
Finding examplars for 11
Task 6/50, range: 0->12, acc: 0.378
Range:  12 14
Test:  1400 Train:  2722
Task 7/50, Epoch 1/70, loss: 0.4009.
Task 7/50, Epoch 2/70, loss: 0.394.
Task 7/50, Epoch 3/70, loss: 0.3905.
Task 7/50, Epoch 4/70, loss: 0.389.
Task 7/50, Epoch 5/70, loss: 0.3877.
Task 7/50, Epoch 6/70, loss: 0.3869.
Task 7/50, Epoch 7/70, loss: 0.3863.
Task 7/50, Epoch 8/70, loss: 0.3861.
Task 7/50, Epoch 9/70, loss: 0.3858.
Task 7/50, Epoch 10/70, loss: 0.3854.
Task 7/50, Epoch 11/70, loss: 0.3852.
Task 7/50, Epoch 12/70, loss: 0.3852.
Task 7/50, Epoch 13/70, loss: 0.3849.
Task 7/50, Epoch 14/70, loss: 0.3847.
Task 7/50, Epoch 15/70, loss: 0.3846.
Task 7/50, Epoch 16/70, loss: 0.3845.
Task 7/50, Epoch 17/70, loss: 0.3843.
Task 7/50, Epoch 18/70, loss

Task 9/50, Epoch 62/70, loss: 0.361.
Task 9/50, Epoch 63/70, loss: 0.3608.
Task 9/50, Epoch 64/70, loss: 0.3606.
Task 9/50, Epoch 65/70, loss: 0.3604.
Task 9/50, Epoch 66/70, loss: 0.3603.
Task 9/50, Epoch 67/70, loss: 0.3601.
Task 9/50, Epoch 68/70, loss: 0.3599.
Task 9/50, Epoch 69/70, loss: 0.3597.
Task 9/50, Epoch 70/70, loss: 0.3595.
Memory stuff up to 18
Finding examplars for 16
Finding examplars for 17
Task 9/50, range: 0->18, acc: 0.357
Range:  18 20
Test:  2000 Train:  2916
Task 10/50, Epoch 1/70, loss: 0.3688.
Task 10/50, Epoch 2/70, loss: 0.3637.
Task 10/50, Epoch 3/70, loss: 0.3615.
Task 10/50, Epoch 4/70, loss: 0.36.
Task 10/50, Epoch 5/70, loss: 0.3591.
Task 10/50, Epoch 6/70, loss: 0.3583.
Task 10/50, Epoch 7/70, loss: 0.3576.
Task 10/50, Epoch 8/70, loss: 0.3573.
Task 10/50, Epoch 9/70, loss: 0.357.
Task 10/50, Epoch 10/70, loss: 0.3566.
Task 10/50, Epoch 11/70, loss: 0.3562.
Task 10/50, Epoch 12/70, loss: 0.3561.
Task 10/50, Epoch 13/70, loss: 0.3558.
Task 10/50, Epoch

Task 12/50, Epoch 53/70, loss: 0.3268.
Task 12/50, Epoch 54/70, loss: 0.3266.
Task 12/50, Epoch 55/70, loss: 0.3264.
Task 12/50, Epoch 56/70, loss: 0.3262.
Task 12/50, Epoch 57/70, loss: 0.326.
Task 12/50, Epoch 58/70, loss: 0.3258.
Task 12/50, Epoch 59/70, loss: 0.3256.
Task 12/50, Epoch 60/70, loss: 0.3254.
Task 12/50, Epoch 61/70, loss: 0.3252.
Task 12/50, Epoch 62/70, loss: 0.3251.
Task 12/50, Epoch 63/70, loss: 0.3249.
Task 12/50, Epoch 64/70, loss: 0.3247.
Task 12/50, Epoch 65/70, loss: 0.3245.
Task 12/50, Epoch 66/70, loss: 0.3244.
Task 12/50, Epoch 67/70, loss: 0.3242.
Task 12/50, Epoch 68/70, loss: 0.3241.
Task 12/50, Epoch 69/70, loss: 0.3239.
Task 12/50, Epoch 70/70, loss: 0.3238.
Memory stuff up to 24
Finding examplars for 22
Finding examplars for 23
Task 12/50, range: 0->24, acc: 0.401
Range:  24 26
Test:  2600 Train:  2989
Task 13/50, Epoch 1/70, loss: 0.3312.
Task 13/50, Epoch 2/70, loss: 0.327.
Task 13/50, Epoch 3/70, loss: 0.3246.
Task 13/50, Epoch 4/70, loss: 0.3233.


...

Task 29/50, Epoch 68/70, loss: 0.1787.
Task 29/50, Epoch 69/70, loss: 0.1787.
Task 29/50, Epoch 70/70, loss: 0.1786.
Memory stuff up to 58
Finding examplars for 56
Finding examplars for 57
Task 29/50, range: 0->58, acc: 0.384
Range:  58 60
Test:  6000 Train:  2972
Task 30/50, Epoch 1/70, loss: 0.1871.
Task 30/50, Epoch 2/70, loss: 0.1843.
Task 30/50, Epoch 3/70, loss: 0.1829.
Task 30/50, Epoch 4/70, loss: 0.1823.
Task 30/50, Epoch 5/70, loss: 0.1818.
Task 30/50, Epoch 6/70, loss: 0.1813.
Task 30/50, Epoch 7/70, loss: 0.1808.
Task 30/50, Epoch 8/70, loss: 0.1804.
Task 30/50, Epoch 9/70, loss: 0.1801.
Task 30/50, Epoch 10/70, loss: 0.1799.
Task 30/50, Epoch 11/70, loss: 0.1796.
Task 30/50, Epoch 12/70, loss: 0.1795.
Task 30/50, Epoch 13/70, loss: 0.1793.
Task 30/50, Epoch 14/70, loss: 0.1792.
Task 30/50, Epoch 15/70, loss: 0.179.
Task 30/50, Epoch 16/70, loss: 0.1788.
Task 30/50, Epoch 17/70, loss: 0.1787.
Task 30/50, Epoch 18/70, loss: 0.1785.
Task 30/50, Epoch 19/70, loss: 0.1784.
Task

Task 32/50, Epoch 58/70, loss: 0.1779.
Task 32/50, Epoch 59/70, loss: 0.1779.
Task 32/50, Epoch 60/70, loss: 0.1778.
Task 32/50, Epoch 61/70, loss: 0.1777.
Task 32/50, Epoch 62/70, loss: 0.1776.
Task 32/50, Epoch 63/70, loss: 0.1776.
Task 32/50, Epoch 64/70, loss: 0.1775.
Task 32/50, Epoch 65/70, loss: 0.1774.
Task 32/50, Epoch 66/70, loss: 0.1774.
Task 32/50, Epoch 67/70, loss: 0.1773.
Task 32/50, Epoch 68/70, loss: 0.1773.
Task 32/50, Epoch 69/70, loss: 0.1772.
Task 32/50, Epoch 70/70, loss: 0.1771.
Memory stuff up to 64
Finding examplars for 62
Finding examplars for 63
Task 32/50, range: 0->64, acc: 0.371
Range:  64 66
Test:  6600 Train:  2984
Task 33/50, Epoch 1/70, loss: 0.1858.
Task 33/50, Epoch 2/70, loss: 0.1829.
Task 33/50, Epoch 3/70, loss: 0.1814.
Task 33/50, Epoch 4/70, loss: 0.1805.
Task 33/50, Epoch 5/70, loss: 0.1797.
Task 33/50, Epoch 6/70, loss: 0.1791.
Task 33/50, Epoch 7/70, loss: 0.1786.
Task 33/50, Epoch 8/70, loss: 0.1783.
Task 33/50, Epoch 9/70, loss: 0.178.
Task

Task 35/50, Epoch 48/70, loss: 0.1709.
Task 35/50, Epoch 49/70, loss: 0.1708.
Task 35/50, Epoch 50/70, loss: 0.1708.
Task 35/50, Epoch 51/70, loss: 0.1707.
Task 35/50, Epoch 52/70, loss: 0.1706.
Task 35/50, Epoch 53/70, loss: 0.1706.
Task 35/50, Epoch 54/70, loss: 0.1705.
Task 35/50, Epoch 55/70, loss: 0.1704.
Task 35/50, Epoch 56/70, loss: 0.1703.
Task 35/50, Epoch 57/70, loss: 0.1703.
Task 35/50, Epoch 58/70, loss: 0.1702.
Task 35/50, Epoch 59/70, loss: 0.1701.
Task 35/50, Epoch 60/70, loss: 0.1701.
Task 35/50, Epoch 61/70, loss: 0.17.
Task 35/50, Epoch 62/70, loss: 0.17.
Task 35/50, Epoch 63/70, loss: 0.1699.
Task 35/50, Epoch 64/70, loss: 0.1698.
Task 35/50, Epoch 65/70, loss: 0.1698.
Task 35/50, Epoch 66/70, loss: 0.1697.
Task 35/50, Epoch 67/70, loss: 0.1697.
Task 35/50, Epoch 68/70, loss: 0.1696.
Task 35/50, Epoch 69/70, loss: 0.1696.
Task 35/50, Epoch 70/70, loss: 0.1695.
Memory stuff up to 70
Finding examplars for 68
Finding examplars for 69
Task 35/50, range: 0->70, acc: 0.35

Task 39/50, Epoch 3/70, loss: 0.165.
Task 39/50, Epoch 4/70, loss: 0.1645.
Task 39/50, Epoch 5/70, loss: 0.1644.
Task 39/50, Epoch 6/70, loss: 0.1642.
Task 39/50, Epoch 7/70, loss: 0.164.
Task 39/50, Epoch 8/70, loss: 0.1638.
Task 39/50, Epoch 9/70, loss: 0.1636.
Task 39/50, Epoch 10/70, loss: 0.1635.
Task 39/50, Epoch 11/70, loss: 0.1633.
Task 39/50, Epoch 12/70, loss: 0.1631.
Task 39/50, Epoch 13/70, loss: 0.163.
Task 39/50, Epoch 14/70, loss: 0.1628.
Task 39/50, Epoch 15/70, loss: 0.1627.
Task 39/50, Epoch 16/70, loss: 0.1626.
Task 39/50, Epoch 17/70, loss: 0.1625.
Task 39/50, Epoch 18/70, loss: 0.1624.
Task 39/50, Epoch 19/70, loss: 0.1624.
Task 39/50, Epoch 20/70, loss: 0.1624.
Task 39/50, Epoch 21/70, loss: 0.1625.
Task 39/50, Epoch 22/70, loss: 0.1624.
Task 39/50, Epoch 23/70, loss: 0.1623.
Task 39/50, Epoch 24/70, loss: 0.1622.
Task 39/50, Epoch 25/70, loss: 0.1621.
Task 39/50, Epoch 26/70, loss: 0.1621.
Task 39/50, Epoch 27/70, loss: 0.162.
Task 39/50, Epoch 28/70, loss: 0.162

Task 41/50, Epoch 67/70, loss: 0.1533.
Task 41/50, Epoch 68/70, loss: 0.1533.
Task 41/50, Epoch 69/70, loss: 0.1533.
Task 41/50, Epoch 70/70, loss: 0.1532.
Memory stuff up to 82
Finding examplars for 80
Finding examplars for 81
Task 41/50, range: 0->82, acc: 0.328
Range:  82 84
Test:  8400 Train:  2968
Task 42/50, Epoch 1/70, loss: 0.1609.
Task 42/50, Epoch 2/70, loss: 0.1594.
Task 42/50, Epoch 3/70, loss: 0.1586.
Task 42/50, Epoch 4/70, loss: 0.1583.
Task 42/50, Epoch 5/70, loss: 0.1581.
Task 42/50, Epoch 6/70, loss: 0.1578.
Task 42/50, Epoch 7/70, loss: 0.1576.
Task 42/50, Epoch 8/70, loss: 0.1574.
Task 42/50, Epoch 9/70, loss: 0.1572.
Task 42/50, Epoch 10/70, loss: 0.1572.
Task 42/50, Epoch 11/70, loss: 0.1571.
Task 42/50, Epoch 12/70, loss: 0.157.
Task 42/50, Epoch 13/70, loss: 0.1569.
Task 42/50, Epoch 14/70, loss: 0.1568.
Task 42/50, Epoch 15/70, loss: 0.1566.
Task 42/50, Epoch 16/70, loss: 0.1565.
Task 42/50, Epoch 17/70, loss: 0.1565.
Task 42/50, Epoch 18/70, loss: 0.1564.
Task

In [22]:
icarl_specialseed_2_wd = fun(increment=2, convnet="rebuffi", bias=True)
icarl_specialseed_2_wd

[49, 63] 2.0
Range:  0 2
Test:  200 Train:  1000
Task 1/50, Epoch 1/70, loss: 2.9618.
Task 1/50, Epoch 2/70, loss: 1.6704.
Task 1/50, Epoch 3/70, loss: 1.235.
Task 1/50, Epoch 4/70, loss: 1.0093.
Task 1/50, Epoch 5/70, loss: 0.8649.
Task 1/50, Epoch 6/70, loss: 0.7597.
Task 1/50, Epoch 7/70, loss: 0.6817.
Task 1/50, Epoch 8/70, loss: 0.6251.
Task 1/50, Epoch 9/70, loss: 0.5803.
Task 1/50, Epoch 10/70, loss: 0.5406.
Task 1/50, Epoch 11/70, loss: 0.5062.
Task 1/50, Epoch 12/70, loss: 0.4773.
Task 1/50, Epoch 13/70, loss: 0.4525.
Task 1/50, Epoch 14/70, loss: 0.4316.
Task 1/50, Epoch 15/70, loss: 0.413.
Task 1/50, Epoch 16/70, loss: 0.396.
Task 1/50, Epoch 17/70, loss: 0.3815.
Task 1/50, Epoch 18/70, loss: 0.3676.
Task 1/50, Epoch 19/70, loss: 0.3552.
Task 1/50, Epoch 20/70, loss: 0.3444.
Task 1/50, Epoch 21/70, loss: 0.3346.
Task 1/50, Epoch 22/70, loss: 0.3265.
Task 1/50, Epoch 23/70, loss: 0.3196.
Task 1/50, Epoch 24/70, loss: 0.3144.
Task 1/50, Epoch 25/70, loss: 0.3089.
Task 1/50, Ep

Task 3/50, Epoch 70/70, loss: 0.2525.
Memory stuff up to 6
Finding examplars for 4
Finding examplars for 5
Task 3/50, range: 0->6, acc: 0.755
Range:  6 8
Test:  800 Train:  2335
Task 4/50, Epoch 1/70, loss: 0.3014.
Task 4/50, Epoch 2/70, loss: 0.2995.
Task 4/50, Epoch 3/70, loss: 0.2911.
Task 4/50, Epoch 4/70, loss: 0.2839.
Task 4/50, Epoch 5/70, loss: 0.2804.
Task 4/50, Epoch 6/70, loss: 0.2785.
Task 4/50, Epoch 7/70, loss: 0.2759.
Task 4/50, Epoch 8/70, loss: 0.275.
Task 4/50, Epoch 9/70, loss: 0.2734.
Task 4/50, Epoch 10/70, loss: 0.2713.
Task 4/50, Epoch 11/70, loss: 0.2693.
Task 4/50, Epoch 12/70, loss: 0.2676.
Task 4/50, Epoch 13/70, loss: 0.2662.
Task 4/50, Epoch 14/70, loss: 0.266.
Task 4/50, Epoch 15/70, loss: 0.2651.
Task 4/50, Epoch 16/70, loss: 0.2638.
Task 4/50, Epoch 17/70, loss: 0.2626.
Task 4/50, Epoch 18/70, loss: 0.2616.
Task 4/50, Epoch 19/70, loss: 0.2608.
Task 4/50, Epoch 20/70, loss: 0.26.
Task 4/50, Epoch 21/70, loss: 0.2591.
Task 4/50, Epoch 22/70, loss: 0.2582.

Task 6/50, Epoch 66/70, loss: 0.1819.
Task 6/50, Epoch 67/70, loss: 0.1816.
Task 6/50, Epoch 68/70, loss: 0.1812.
Task 6/50, Epoch 69/70, loss: 0.1808.
Task 6/50, Epoch 70/70, loss: 0.1804.
Memory stuff up to 12
Finding examplars for 10
Finding examplars for 11
Task 6/50, range: 0->12, acc: 0.773
Range:  12 14
Test:  1400 Train:  2972
Task 7/50, Epoch 1/70, loss: 0.2272.
Task 7/50, Epoch 2/70, loss: 0.2246.
Task 7/50, Epoch 3/70, loss: 0.2237.
Task 7/50, Epoch 4/70, loss: 0.2221.
Task 7/50, Epoch 5/70, loss: 0.2218.
Task 7/50, Epoch 6/70, loss: 0.2203.
Task 7/50, Epoch 7/70, loss: 0.2195.
Task 7/50, Epoch 8/70, loss: 0.2181.
Task 7/50, Epoch 9/70, loss: 0.2173.
Task 7/50, Epoch 10/70, loss: 0.2165.
Task 7/50, Epoch 11/70, loss: 0.2158.
Task 7/50, Epoch 12/70, loss: 0.2153.
Task 7/50, Epoch 13/70, loss: 0.2146.
Task 7/50, Epoch 14/70, loss: 0.2138.
Task 7/50, Epoch 15/70, loss: 0.2129.
Task 7/50, Epoch 16/70, loss: 0.2123.
Task 7/50, Epoch 17/70, loss: 0.2117.
Task 7/50, Epoch 18/70, lo

Task 9/50, Epoch 62/70, loss: 0.1828.
Task 9/50, Epoch 63/70, loss: 0.1824.
Task 9/50, Epoch 64/70, loss: 0.1821.
Task 9/50, Epoch 65/70, loss: 0.1818.
Task 9/50, Epoch 66/70, loss: 0.1814.
Task 9/50, Epoch 67/70, loss: 0.1811.
Task 9/50, Epoch 68/70, loss: 0.1808.
Task 9/50, Epoch 69/70, loss: 0.1805.
Task 9/50, Epoch 70/70, loss: 0.1802.
Memory stuff up to 18
Finding examplars for 16
Finding examplars for 17
Task 9/50, range: 0->18, acc: 0.705
Range:  18 20
Test:  2000 Train:  2998
Task 10/50, Epoch 1/70, loss: 0.1939.
Task 10/50, Epoch 2/70, loss: 0.1918.
Task 10/50, Epoch 3/70, loss: 0.1908.
Task 10/50, Epoch 4/70, loss: 0.1896.
Task 10/50, Epoch 5/70, loss: 0.189.
Task 10/50, Epoch 6/70, loss: 0.1888.
Task 10/50, Epoch 7/70, loss: 0.1886.
Task 10/50, Epoch 8/70, loss: 0.188.
Task 10/50, Epoch 9/70, loss: 0.1874.
Task 10/50, Epoch 10/70, loss: 0.1869.
Task 10/50, Epoch 11/70, loss: 0.1863.
Task 10/50, Epoch 12/70, loss: 0.1858.
Task 10/50, Epoch 13/70, loss: 0.1852.
Task 10/50, Epo

Task 12/50, Epoch 53/70, loss: 0.1754.
Task 12/50, Epoch 54/70, loss: 0.1751.
Task 12/50, Epoch 55/70, loss: 0.1748.
Task 12/50, Epoch 56/70, loss: 0.1744.
Task 12/50, Epoch 57/70, loss: 0.1741.
Task 12/50, Epoch 58/70, loss: 0.1738.
Task 12/50, Epoch 59/70, loss: 0.1735.
Task 12/50, Epoch 60/70, loss: 0.1732.
Task 12/50, Epoch 61/70, loss: 0.1729.
Task 12/50, Epoch 62/70, loss: 0.1726.
Task 12/50, Epoch 63/70, loss: 0.1724.
Task 12/50, Epoch 64/70, loss: 0.1721.
Task 12/50, Epoch 65/70, loss: 0.1718.
Task 12/50, Epoch 66/70, loss: 0.1716.
Task 12/50, Epoch 67/70, loss: 0.1713.
Task 12/50, Epoch 68/70, loss: 0.1711.
Task 12/50, Epoch 69/70, loss: 0.1709.
Task 12/50, Epoch 70/70, loss: 0.1706.
Memory stuff up to 24
Finding examplars for 22
Finding examplars for 23
Task 12/50, range: 0->24, acc: 0.663
Range:  24 26
Test:  2600 Train:  2992
Task 13/50, Epoch 1/70, loss: 0.1861.
Task 13/50, Epoch 2/70, loss: 0.1846.
Task 13/50, Epoch 3/70, loss: 0.184.
Task 13/50, Epoch 4/70, loss: 0.1839.

...

Task 41/50, Epoch 27/70, loss: 0.1561.
Task 41/50, Epoch 28/70, loss: 0.1561.
Task 41/50, Epoch 29/70, loss: 0.1561.
Task 41/50, Epoch 30/70, loss: 0.1561.
Task 41/50, Epoch 31/70, loss: 0.1561.
Task 41/50, Epoch 32/70, loss: 0.1561.
Task 41/50, Epoch 33/70, loss: 0.1561.
Task 41/50, Epoch 34/70, loss: 0.1561.
Task 41/50, Epoch 35/70, loss: 0.1561.
Task 41/50, Epoch 36/70, loss: 0.1561.
Task 41/50, Epoch 37/70, loss: 0.1561.
Task 41/50, Epoch 38/70, loss: 0.156.
Task 41/50, Epoch 39/70, loss: 0.156.
Task 41/50, Epoch 40/70, loss: 0.156.
Task 41/50, Epoch 41/70, loss: 0.156.
Task 41/50, Epoch 42/70, loss: 0.156.
Task 41/50, Epoch 43/70, loss: 0.156.
Task 41/50, Epoch 44/70, loss: 0.1559.
Task 41/50, Epoch 45/70, loss: 0.1559.
Task 41/50, Epoch 46/70, loss: 0.1559.
Task 41/50, Epoch 47/70, loss: 0.1559.
Task 41/50, Epoch 48/70, loss: 0.1559.
Task 41/50, Epoch 49/70, loss: 0.1559.
Task 41/50, Epoch 50/70, loss: 0.1559.
Task 41/50, Epoch 51/70, loss: 0.1558.
Task 41/50, Epoch 52/70, loss: 

Task 44/50, Epoch 18/70, loss: 0.1563.
Task 44/50, Epoch 19/70, loss: 0.1563.
Task 44/50, Epoch 20/70, loss: 0.1562.
Task 44/50, Epoch 21/70, loss: 0.1562.
Task 44/50, Epoch 22/70, loss: 0.1561.
Task 44/50, Epoch 23/70, loss: 0.156.
Task 44/50, Epoch 24/70, loss: 0.156.
Task 44/50, Epoch 25/70, loss: 0.1559.
Task 44/50, Epoch 26/70, loss: 0.1559.
Task 44/50, Epoch 27/70, loss: 0.1558.
Task 44/50, Epoch 28/70, loss: 0.1558.
Task 44/50, Epoch 29/70, loss: 0.1557.
Task 44/50, Epoch 30/70, loss: 0.1557.
Task 44/50, Epoch 31/70, loss: 0.1557.
Task 44/50, Epoch 32/70, loss: 0.1557.
Task 44/50, Epoch 33/70, loss: 0.1557.
Task 44/50, Epoch 34/70, loss: 0.1556.
Task 44/50, Epoch 35/70, loss: 0.1556.
Task 44/50, Epoch 36/70, loss: 0.1556.
Task 44/50, Epoch 37/70, loss: 0.1555.
Task 44/50, Epoch 38/70, loss: 0.1555.
Task 44/50, Epoch 39/70, loss: 0.1555.
Task 44/50, Epoch 40/70, loss: 0.1555.
Task 44/50, Epoch 41/70, loss: 0.1555.
Task 44/50, Epoch 42/70, loss: 0.1554.
Task 44/50, Epoch 43/70, lo

Task 47/50, Epoch 9/70, loss: 0.1621.
Task 47/50, Epoch 10/70, loss: 0.1619.
Task 47/50, Epoch 11/70, loss: 0.1617.
Task 47/50, Epoch 12/70, loss: 0.1616.
Task 47/50, Epoch 13/70, loss: 0.1615.
Task 47/50, Epoch 14/70, loss: 0.1614.
Task 47/50, Epoch 15/70, loss: 0.1613.
Task 47/50, Epoch 16/70, loss: 0.1613.
Task 47/50, Epoch 17/70, loss: 0.1612.
Task 47/50, Epoch 18/70, loss: 0.1612.
Task 47/50, Epoch 19/70, loss: 0.1611.
Task 47/50, Epoch 20/70, loss: 0.1611.
Task 47/50, Epoch 21/70, loss: 0.161.
Task 47/50, Epoch 22/70, loss: 0.161.
Task 47/50, Epoch 23/70, loss: 0.161.
Task 47/50, Epoch 24/70, loss: 0.1609.
Task 47/50, Epoch 25/70, loss: 0.1609.
Task 47/50, Epoch 26/70, loss: 0.1608.
Task 47/50, Epoch 27/70, loss: 0.1607.
Task 47/50, Epoch 28/70, loss: 0.1607.
Task 47/50, Epoch 29/70, loss: 0.1606.
Task 47/50, Epoch 30/70, loss: 0.1606.
Task 47/50, Epoch 31/70, loss: 0.1605.
Task 47/50, Epoch 32/70, loss: 0.1605.
Task 47/50, Epoch 33/70, loss: 0.1605.
Task 47/50, Epoch 34/70, loss

Task 49/50, range: 0->98, acc: 0.326
Range:  98 100
Test:  10000 Train:  2960
Task 50/50, Epoch 1/70, loss: 0.1662.
Task 50/50, Epoch 2/70, loss: 0.1653.
Task 50/50, Epoch 3/70, loss: 0.1653.
Task 50/50, Epoch 4/70, loss: 0.1654.
Task 50/50, Epoch 5/70, loss: 0.1654.
Task 50/50, Epoch 6/70, loss: 0.1653.
Task 50/50, Epoch 7/70, loss: 0.1652.
Task 50/50, Epoch 8/70, loss: 0.1652.
Task 50/50, Epoch 9/70, loss: 0.165.
Task 50/50, Epoch 10/70, loss: 0.165.
Task 50/50, Epoch 11/70, loss: 0.1649.
Task 50/50, Epoch 12/70, loss: 0.1648.
Task 50/50, Epoch 13/70, loss: 0.1647.
Task 50/50, Epoch 14/70, loss: 0.1647.
Task 50/50, Epoch 15/70, loss: 0.1646.
Task 50/50, Epoch 16/70, loss: 0.1645.
Task 50/50, Epoch 17/70, loss: 0.1644.
Task 50/50, Epoch 18/70, loss: 0.1643.
Task 50/50, Epoch 19/70, loss: 0.1644.
Task 50/50, Epoch 20/70, loss: 0.1644.
Task 50/50, Epoch 21/70, loss: 0.1643.
Task 50/50, Epoch 22/70, loss: 0.1643.
Task 50/50, Epoch 23/70, loss: 0.1643.
Task 50/50, Epoch 24/70, loss: 0.164

[0.96,
 0.825,
 0.755,
 0.77,
 0.761,
 0.7733333333333333,
 0.7485714285714286,
 0.735,
 0.705,
 0.6925,
 0.6695454545454546,
 0.6629166666666667,
 0.6380769230769231,
 0.6435714285714286,
 0.6256666666666667,
 0.62375,
 0.5982352941176471,
 0.5838888888888889,
 0.5686842105263158,
 0.555,
 0.5504761904761905,
 0.5436363636363636,
 0.5371739130434783,
 0.5316666666666666,
 0.508,
 0.5098076923076923,
 0.5007407407407407,
 0.5017857142857143,
 0.4956896551724138,
 0.49033333333333334,
 0.4832258064516129,
 0.46890625,
 0.45621212121212124,
 0.44691176470588234,
 0.4422857142857143,
 0.4409722222222222,
 0.43067567567567566,
 0.42407894736842106,
 0.4146153846153846,
 0.400125,
 0.39890243902439027,
 0.39261904761904765,
 0.37651162790697673,
 0.3785227272727273,
 0.36866666666666664,
 0.3521739130434783,
 0.34691489361702127,
 0.3338541666666667,
 0.32612244897959186,
 0.3193]